In [7]:
# from bb_behavior import plot
from bb_behavior import db as bbdb
import bb_behavior
import pandas as pd
import os
import sys
sys.path.append('/home/mi/rrszynka/mnt/janek/Beesbook-janek/Python-modules/') #For bee_helpers, file_helpers and cache
from bee_cache import Cache, CacheType, CacheFormat; c = Cache()
import datetime

import numpy as np
from tqdm import tqdm

/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
gaps = c.load('with_coordinates', format = CacheFormat.hdf)

In [ ]:
gaps.head()

In [ ]:
#Add frame to the df 
gaps_sample = gaps.sample(15)

In [ ]:
gaps.sample(15).timestamp[0].timestamp()

In [ ]:
frame_ids = gaps_sample.apply(process_row, axis=1)

In [ ]:
gaps_sample = gaps_sample.assign(frame_id=frame_ids).dropna()

In [ ]:
in_data = [(row.bee_id, row.frame_id, row.timestamp) for _, row in gaps_sample.iterrows()]

In [ ]:
for (bee, frame, timestamp) in in_data:
    name = str(frame)+'_' + str(bee)
    video = plot.plot_bees(bee_ids=[bee], 
                       frame_id=frame, frame_margin=20,
                       path_alpha=None, bt_export=None, # '/home/mi/rrszynka/mnt/janek/caches/Videos/bt_export/'+name+".json",
                       plot_labels = True, plot_markers=True)

    for frame in video._frames:
        frame._title = "\n exit: " + str(timestamp)
    
    video._crop_margin = None
    video.get_video(save_to_path='/home/mi/rrszynka/mnt/janek/caches/Videos/' + name + "t2.mp4")
    
#TODO:
# - get a fullscreen vid? or are the small ones better? 
# - what's the size of fullscreen vs small?

# - how do you get centered videos?
# "if the passed bee is not in the frame, it will be "
# if bt_export is off?

In [ ]:
bee

In [8]:
bee = 3006 
frame = 12373576517183002035
timestamp = datetime.datetime(2016, 8, 16, 4, 59, 30)

In [7]:
name = str(frame)+'_' + str(bee)

In [ ]:
video = plot.plot_bees(bee_ids=[bee], 
                   frame_id=frame, frame_margin=10,
                   path_alpha=None, bt_export=None,
                   plot_labels = True, plot_markers=True)

In [ ]:
for frame in video._frames:
    frame._title = "\ntest"

In [9]:
get_frame_id_for_bee_id_and_timestamp(bee, timestamp)

Decimal('12373576517183002035')

In [9]:
type(bee)

int

In [10]:
type(timestamp)

datetime.datetime

In [ ]:
video.get_video(save_to_path='/home/mi/rrszynka/mnt/janek/caches/Videos/' + name + "-test.mp4")

In [5]:
def process_row(row):
    return get_frame_id_for_bee_id_and_timestamp(row['bee_id'], row['timestamp'])

#TODO: analyze how this works 
def get_frame_id_for_bee_id_and_timestamp(bee_id, dt):
    for cam_id in range(4): 
        frames = bbdb.get_frames(cam_id, dt.timestamp(), dt.timestamp() + 5)
        if len(frames) == 0:
            continue
        detections = bbdb.get_bee_detections(bee_id, frames=frames)
        for d in detections:
            if d is not None:
                return d[1]

In [ ]:
det = c.load("DETECTIONS-2016-08-13_15:00:00", type = CacheType.detections, format = CacheFormat.csv)

In [ ]:
det[det.cam_id==0].head(1)

In [ ]:
samples = []

for cam in range(4):
    samples.append(det[det.cam_id==cam].head(1))